# Keras-RL DQN Model


At first we will import all necessary packages:

In [2]:
import time  # to reduce the game speed when playing manually

import gym  # Contains the game we want to play
from pyglet.window import key  # for manual playing

# import necessary blocks from keras to build the Deep Learning backbone of our agent
from tensorflow.keras.models import Sequential  # To compose multiple Layers
from tensorflow.keras.layers import Dense  # Fully-Connected layer
from tensorflow.keras.layers import Activation  # Activation functions
from tensorflow.keras.layers import Flatten  # Flatten function

from tensorflow.keras.optimizers import Adam  # Adam optimizer

# Now the keras-rl2 agent. Dont get confused as it is only called rl and not keras-rl

from rl.agents.dqn import DQNAgent  # Use the basic Deep-Q-Network agent

In [3]:
import wandb
from wandb.keras import WandbCallback

wandb.init(config={"hyper": "parameter"})

wandb: Currently logged in as: skyfall-blue (use `wandb login --relogin` to force relogin)


Now we will create the environment:

In [4]:
env_name = ENV_NAME = 'LunarLander-v2'  # https://gym.openai.com/envs/LunarLander-v2/
env = gym.make(env_name)  # create the environment
nb_actions = env.action_space.n  # get the number of possible actions
NUMBER_STEPS = 150000

Lets watch how the game looks when chosing random actions

In [ ]:
env.reset()  # reset the environment to the initial state
for _ in range(200):  # play for max 200 iterations
    env.render(mode="human")  # render the current game state on your screen
    random_action = env.action_space.sample()  # chose a random action
    env.step(random_action)  # execute that action
env.close()  # close the environment

In [ ]:
action = 0
def key_press(k, mod):
    '''
    This function gets the key press for gym
    '''
    global action
    if k == key.LEFT:
        action = 0
    if k == key.RIGHT:
        action = 1

env.reset()
rewards = 0
for _ in range(1000):
    env.render(mode="human")
    env.viewer.window.on_key_press = key_press  # update the key press
    observation, reward, done, info = env.step(action)
    rewards+=1
    if done:
        print(f"You got {rewards} points!")
        break
    time.sleep(0.1)  # reduce speed a little bit
env.close()

**TASK: Create the Neural Network for your Deep-Q-Agent**
Take a look at the size of the action space and the size of the observation space.
You are free to chose any architecture you want!
Hint: It already works with three layers, each having 64 neurons.

In [5]:
model = Sequential()
# https://keras.io/api/layers/reshaping_layers/flatten/
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))

model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(32))
model.add(Activation('relu'))

model.add(Dense(nb_actions))
model.add(Activation('linear'))

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 8)                 0         
_________________________________________________________________
dense (Dense)                (None, 64)                576       
_________________________________________________________________
activation (Activation)      (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
activation_1 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
activation_2 (Activation)    (None, 32)                0

Lets create the DQN agent from keras-rl
For this setting, the agent takes the following parameters:

1. model = The model
2. nb_actions = The number of actions (2 in this case)
3. memory = The action replay memory. You can choose between the *SequentialMemory()* and *EpisodeParameterMemory() which is only used for one RL agent called CEM*
4. nb_steps_warmup = How many iterations without training - Used to fill the memory
5. target_model_update = When do we update the target model?
6. Action Selection policy. You can choose between a *LinearAnnealedPolicy()*, *SoftmaxPolicy()*, *EpsGreedyQPolicy()*, *GreedyQPolicy()*, *GreedyQPolicy()*, *MaxBoltzmannQPolicy()* and *BoltzmannGumbelQPolicy()*. We use all of them during the next notebooks but feel free to try them out and inspect which works best here

There are some more parameters, you can pass to the DQN Agent. Feel free to explore them, but we will also take a look at them together in the remaining notebooks

Here we initialize the circular buffer with a limit of 50000 and a window length of 1.
The window length describes the number of subsequent actions stored for a state.
This will be demonstrated in the next lecture, when we start dealing with images

In [6]:
from rl.memory import SequentialMemory  # Sequential Memory for storing observations ( optimized circular buffer)

memory = SequentialMemory(limit=50000, window_length=1)


Then we define the Action Selection Policy: <br />
We use *LinearAnnealedPolicy* in order to perform the epsilon greedy strategy with decaying epsilon. <br />
*LinearAnnealedPolicy* accepts an action selection policy, its maximal and minimal values and a step number in order to create a dynimal policy. <br/>
The minimal value epsilon can reach during training is 0.1.<br />
For evaluation (e.g running the agent) it is fixed to 0


In [7]:
# LinearAnnealedPolicy allows to decay the epsilon for the epsilon greedy strategy
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), 
                              attr='eps',
                              value_max=1.,
                              value_min=.1,
                              value_test=0,
                              nb_steps=NUMBER_STEPS) 


**TASK: Create the DQNAgent** <br />
Feel free to play with the nb_steps_warump, target_model_update, batch_size and gamma parameters. <br />
Hint:<br />
You can try *nb_steps_warmup*=1000, *target_model_update*=1000, *batch_size*=32 and *gamma*=0.99 as a first guess

In [8]:
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10,
               target_model_update=100, policy=policy, batch_size=64, gamma=0.991)


Finally we compile our model with the Adam optimizer and a learning rate of 0.001.<br />
We log the Mean Absolute Error

In [9]:
# Use learning_rate instead of lr if you get warning
dqn.compile(Adam(lr=0.001), metrics=['mse']) 

Now we run the training for 150000 steps. You can change visualize=True if you want to watch your model learning.
Keep in mind that this increases the running time
The training time is around 5 min so grep your favorite beverage and stay tuned


In [13]:
dqn.fit(env, nb_steps=NUMBER_STEPS, visualize=False, verbose=2, callbacks=[WandbCallback()])

Training for 150000 steps ...
    101/150000: episode: 1, duration: 0.680s, episode steps: 101, steps per second: 148, episode reward: -80.514, mean reward: -0.797 [-100.000,  6.883], mean action: 1.554 [0.000, 3.000],  loss: 12.809403, mse: 5478.669884, mean_q: 74.996231, mean_eps: 0.999667
    192/150000: episode: 2, duration: 0.730s, episode steps:  91, steps per second: 125, episode reward: -69.796, mean reward: -0.767 [-100.000, 16.831], mean action: 1.593 [0.000, 3.000],  loss: 8.018393, mse: 5453.873549, mean_q: 74.532044, mean_eps: 0.999124
    262/150000: episode: 3, duration: 0.496s, episode steps:  70, steps per second: 141, episode reward: -78.401, mean reward: -1.120 [-100.000,  7.790], mean action: 1.486 [0.000, 3.000],  loss: 18.678165, mse: 5459.173535, mean_q: 74.923272, mean_eps: 0.998641
    340/150000: episode: 4, duration: 0.649s, episode steps:  78, steps per second: 120, episode reward: -99.475, mean reward: -1.275 [-100.000, 15.132], mean action: 1.590 [0.000, 3

   2933/150000: episode: 32, duration: 0.969s, episode steps: 102, steps per second: 105, episode reward: -321.955, mean reward: -3.156 [-100.000,  7.311], mean action: 1.382 [0.000, 3.000],  loss: 14.968071, mse: 5121.941550, mean_q: 71.737554, mean_eps: 0.982711
   3037/150000: episode: 33, duration: 1.008s, episode steps: 104, steps per second: 103, episode reward: -271.764, mean reward: -2.613 [-100.000,  5.886], mean action: 1.365 [0.000, 3.000],  loss: 14.510038, mse: 5327.710977, mean_q: 73.188263, mean_eps: 0.982093
   3132/150000: episode: 34, duration: 0.866s, episode steps:  95, steps per second: 110, episode reward: -222.916, mean reward: -2.346 [-100.000,  5.440], mean action: 1.568 [0.000, 3.000],  loss: 11.553653, mse: 5323.510537, mean_q: 73.596199, mean_eps: 0.981496
   3223/150000: episode: 35, duration: 0.740s, episode steps:  91, steps per second: 123, episode reward: -455.181, mean reward: -5.002 [-100.000,  0.088], mean action: 1.736 [0.000, 3.000],  loss: 5.78367

   5866/150000: episode: 63, duration: 0.811s, episode steps: 113, steps per second: 139, episode reward: -337.795, mean reward: -2.989 [-100.000,  1.207], mean action: 1.407 [0.000, 3.000],  loss: 11.163018, mse: 4855.290795, mean_q: 68.276288, mean_eps: 0.965146
   5928/150000: episode: 64, duration: 0.440s, episode steps:  62, steps per second: 141, episode reward: -85.872, mean reward: -1.385 [-100.000, 11.943], mean action: 1.661 [0.000, 3.000],  loss: 6.871792, mse: 4865.794564, mean_q: 69.683473, mean_eps: 0.964621
   6036/150000: episode: 65, duration: 0.774s, episode steps: 108, steps per second: 139, episode reward: -118.802, mean reward: -1.100 [-100.000,  6.268], mean action: 1.694 [0.000, 3.000],  loss: 13.300338, mse: 4664.253863, mean_q: 66.745900, mean_eps: 0.964111
   6139/150000: episode: 66, duration: 0.714s, episode steps: 103, steps per second: 144, episode reward: -433.573, mean reward: -4.209 [-100.000,  0.642], mean action: 1.437 [0.000, 3.000],  loss: 9.567291,

   8705/150000: episode: 94, duration: 0.627s, episode steps:  80, steps per second: 128, episode reward: -54.952, mean reward: -0.687 [-100.000, 105.737], mean action: 1.387 [0.000, 3.000],  loss: 16.828836, mse: 5084.430768, mean_q: 68.814553, mean_eps: 0.948013
   8827/150000: episode: 95, duration: 0.897s, episode steps: 122, steps per second: 136, episode reward: -161.783, mean reward: -1.326 [-100.000,  4.932], mean action: 1.566 [0.000, 3.000],  loss: 11.224929, mse: 4800.568750, mean_q: 66.303100, mean_eps: 0.947407
   8937/150000: episode: 96, duration: 0.733s, episode steps: 110, steps per second: 150, episode reward: -518.726, mean reward: -4.716 [-100.000, 50.564], mean action: 1.636 [0.000, 3.000],  loss: 19.265038, mse: 4708.990039, mean_q: 65.103234, mean_eps: 0.946711
   9049/150000: episode: 97, duration: 0.772s, episode steps: 112, steps per second: 145, episode reward: -53.647, mean reward: -0.479 [-100.000, 10.867], mean action: 1.500 [0.000, 3.000],  loss: 14.42554

  11522/150000: episode: 125, duration: 0.853s, episode steps: 115, steps per second: 135, episode reward: -32.501, mean reward: -0.283 [-100.000, 87.846], mean action: 1.348 [0.000, 3.000],  loss: 17.560802, mse: 4696.684258, mean_q: 64.767067, mean_eps: 0.931216
  11601/150000: episode: 126, duration: 0.609s, episode steps:  79, steps per second: 130, episode reward: -101.281, mean reward: -1.282 [-100.000, 11.960], mean action: 1.544 [0.000, 3.000],  loss: 16.239678, mse: 4667.732558, mean_q: 63.910011, mean_eps: 0.930634
  11693/150000: episode: 127, duration: 0.722s, episode steps:  92, steps per second: 127, episode reward: -212.637, mean reward: -2.311 [-100.000, 21.576], mean action: 1.533 [0.000, 3.000],  loss: 14.052078, mse: 4767.283187, mean_q: 64.869002, mean_eps: 0.930121
  11804/150000: episode: 128, duration: 1.220s, episode steps: 111, steps per second:  91, episode reward: -41.814, mean reward: -0.377 [-100.000, 89.212], mean action: 1.532 [0.000, 3.000],  loss: 19.84

...

  98366/150000: episode: 838, duration: 8.314s, episode steps: 1000, steps per second: 120, episode reward: 50.083, mean reward:  0.050 [-23.751, 21.896], mean action: 1.576 [0.000, 3.000],  loss: 42.329394, mse: 25468.189191, mean_q: 162.577027, mean_eps: 0.412807
  98476/150000: episode: 839, duration: 0.874s, episode steps: 110, steps per second: 126, episode reward: -19.273, mean reward: -0.175 [-100.000, 23.133], mean action: 1.664 [0.000, 3.000],  loss: 34.415024, mse: 24989.572496, mean_q: 160.581412, mean_eps: 0.409477
  99476/150000: episode: 840, duration: 7.996s, episode steps: 1000, steps per second: 125, episode reward: -27.758, mean reward: -0.028 [-22.987, 22.546], mean action: 1.503 [0.000, 3.000],  loss: 39.856778, mse: 24313.945555, mean_q: 157.457801, mean_eps: 0.406147
  99554/150000: episode: 841, duration: 0.544s, episode steps:  78, steps per second: 143, episode reward: 18.464, mean reward:  0.237 [-100.000, 17.191], mean action: 1.756 [0.000, 3.000],  loss: 40.

 107338/150000: episode: 869, duration: 0.785s, episode steps:  90, steps per second: 115, episode reward: -31.293, mean reward: -0.348 [-100.000, 11.018], mean action: 1.833 [0.000, 3.000],  loss: 38.369446, mse: 16821.617459, mean_q: 130.001409, mean_eps: 0.356245
 107525/150000: episode: 870, duration: 1.347s, episode steps: 187, steps per second: 139, episode reward: -323.132, mean reward: -1.728 [-100.000,  5.225], mean action: 1.663 [0.000, 3.000],  loss: 35.240715, mse: 16826.054672, mean_q: 129.878383, mean_eps: 0.355414
 107667/150000: episode: 871, duration: 0.982s, episode steps: 142, steps per second: 145, episode reward: 17.470, mean reward:  0.123 [-100.000, 17.696], mean action: 1.641 [0.000, 3.000],  loss: 29.149170, mse: 17246.226363, mean_q: 132.406731, mean_eps: 0.354427
 107772/150000: episode: 872, duration: 0.735s, episode steps: 105, steps per second: 143, episode reward: -8.210, mean reward: -0.078 [-100.000, 13.594], mean action: 1.410 [0.000, 3.000],  loss: 27

 115379/150000: episode: 900, duration: 1.531s, episode steps: 187, steps per second: 122, episode reward: -38.069, mean reward: -0.204 [-100.000, 14.830], mean action: 1.759 [0.000, 3.000],  loss: 29.214248, mse: 12479.490600, mean_q: 111.339062, mean_eps: 0.308290
 115508/150000: episode: 901, duration: 1.066s, episode steps: 129, steps per second: 121, episode reward: -6.838, mean reward: -0.053 [-100.000, 16.441], mean action: 1.574 [0.000, 3.000],  loss: 21.236280, mse: 12362.058791, mean_q: 110.638418, mean_eps: 0.307342
 116030/150000: episode: 902, duration: 4.125s, episode steps: 522, steps per second: 127, episode reward: -246.482, mean reward: -0.472 [-100.000, 23.248], mean action: 1.402 [0.000, 3.000],  loss: 25.533646, mse: 11798.188718, mean_q: 107.390052, mean_eps: 0.305389
 116166/150000: episode: 903, duration: 1.004s, episode steps: 136, steps per second: 135, episode reward: -86.716, mean reward: -0.638 [-100.000,  4.113], mean action: 1.559 [0.000, 3.000],  loss: 2

 129932/150000: episode: 931, duration: 2.622s, episode steps: 369, steps per second: 141, episode reward: 271.219, mean reward:  0.735 [-18.292, 100.000], mean action: 1.545 [0.000, 3.000],  loss: 16.001201, mse: 6165.214602, mean_q: 77.624931, mean_eps: 0.221518
 130932/150000: episode: 932, duration: 7.584s, episode steps: 1000, steps per second: 132, episode reward: 90.525, mean reward:  0.091 [-23.842, 17.712], mean action: 1.196 [0.000, 3.000],  loss: 16.799092, mse: 5900.092398, mean_q: 74.894335, mean_eps: 0.217411
 131932/150000: episode: 933, duration: 8.037s, episode steps: 1000, steps per second: 124, episode reward: 52.146, mean reward:  0.052 [-17.764, 19.270], mean action: 1.361 [0.000, 3.000],  loss: 16.380088, mse: 5558.705601, mean_q: 73.146607, mean_eps: 0.211411
 132024/150000: episode: 934, duration: 0.652s, episode steps:  92, steps per second: 141, episode reward: 30.281, mean reward:  0.329 [-100.000, 12.378], mean action: 1.837 [0.000, 3.000],  loss: 15.097853,

 143297/150000: episode: 962, duration: 7.407s, episode steps: 898, steps per second: 121, episode reward: 152.072, mean reward:  0.169 [-22.902, 100.000], mean action: 1.199 [0.000, 3.000],  loss: 15.533905, mse: 3977.129760, mean_q: 62.673222, mean_eps: 0.142915
 143714/150000: episode: 963, duration: 3.113s, episode steps: 417, steps per second: 134, episode reward: 156.957, mean reward:  0.376 [-18.231, 100.000], mean action: 2.149 [0.000, 3.000],  loss: 14.468026, mse: 3941.503255, mean_q: 62.665976, mean_eps: 0.138970
 144284/150000: episode: 964, duration: 4.540s, episode steps: 570, steps per second: 126, episode reward: 187.628, mean reward:  0.329 [-21.158, 100.000], mean action: 1.454 [0.000, 3.000],  loss: 15.433039, mse: 4042.094885, mean_q: 63.502053, mean_eps: 0.136009
 145103/150000: episode: 965, duration: 6.069s, episode steps: 819, steps per second: 135, episode reward: 250.648, mean reward:  0.306 [-18.309, 100.000], mean action: 1.004 [0.000, 3.000],  loss: 16.8608

Wow! After only some minutes of training, we achieve great results!
The reason for this is, that keras-rl has implemented many optimization strategies (e.g the optimized replay buffer) which lead to a much faster convergence than our DQN implemented by hand

In [ ]:
# After training is done, we save the final weights.
dqn.save_weights(f'dqn_{env_name}_weights_0,995_a0,001.h5f', overwrite=True)

In [ ]:
# Finally, evaluate our algorithm for 5 episodes.
dqn.test(env, nb_episodes=200, visualize=False, callbacks=[WandbCallback()])
env.close()

In [ ]:
# Finally, evaluate our algorithm for 5 episodes.
dqn.test(env, nb_episodes=5, visualize=True)
env.close()

Testing for 5 episodes ...
Episode 1: reward: 250.716, steps: 321
Episode 2: reward: 257.224, steps: 242
